In [ ]:
import torch
from torchaudio import list_audio_backends
from ultralytics import YOLO
import supervision as sv
import numpy as np
import cv2
from tracker import Tracker
from detector import Detector
from teams import TeamAffiliation

In [ ]:
model = YOLO("runs/detect/train38/weights/best.pt")

In [ ]:
tracker = sv.ByteTrack()

In [ ]:
annotator = sv.BoxAnnotator()

In [ ]:
def load_frames(source_path: str) -> list[np.ndarray]:
    frames: list[np.ndarray] = list()
    cap = cv2.VideoCapture(source_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames

In [ ]:
video = load_frames(r"C:\Users\Marcin\Videos\ganzgut.mp4")

In [ ]:
def detect(frames: list[np.ndarray], batch_size: int = 40) -> list:
    detections = list()
    for frame in frames:
        results = model.predict(frame, conf=0.5)
        detections += results
    return detections

In [ ]:
detections = detect(video)

In [ ]:
detections

In [ ]:
trckr = sv.ByteTrack()
dets = sv.Detections.from_ultralytics(detections)

In [ ]:
def track(frames: list[np.ndarray]) -> list:
    detections = detect(frames)
    tracker = sv.ByteTrack()
    tracks: list = list()
    for i, detection in enumerate(detections):
        dets = sv.Detections.from_ultralytics(detection)
        tracks.append(tracker.update_with_detections(dets))
    return tracks

In [ ]:
trackers = track(video)

In [ ]:
trackers[0]

In [ ]:
list(track.tracker_id for track in trackers[:10])

In [ ]:
def anotate_frames(frames: list[np.ndarray], trackers: list) -> list[np.ndarray]:
    for frame, tracker in zip(frames, trackers):
        for bbox, tracker_id in zip(tracker.xyxy, tracker.tracker_id):
            x1, y1, x2, y2 = map(int, bbox)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, str(tracker_id), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 4)
    return frames

In [ ]:
annotated = anotate_frames(video, trackers)

In [ ]:
# przeliczenie na homografie tam wyznaczenie trasy i potem przeliczenie z powrotem dla każdej klatki
def draw_traces(frames: list[np.ndarray], trackers: list) -> list[np.ndarray]:
    tracked_player_id = 4
    tracker_id = trackers[0].tracker_id[tracked_player_id]
    trace = list()
    for frame, tracker in zip(frames, trackers):
        try:
            tracker_list_pos = int(np.where(tracker.tracker_id == tracker_id)[0][0])
        except IndexError:
            break
        tracked_player_bbox = tracker.xyxy[tracker_list_pos]
        x = int((tracked_player_bbox[0] + tracked_player_bbox[2]) / 2)
        y = int((tracked_player_bbox[1] + tracked_player_bbox[3]) / 2)
        trace.append((x, y))
        for x, y in trace:
            cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
    return frames

In [ ]:
traces = draw_traces(video, trackers)

In [ ]:
traces[0].shape

In [ ]:
cv2.imshow('rsssss', traces[50])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
out = cv2.VideoWriter(r"C:\Users\Marcin\Videos\ganzgut_track.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 25, (1280, 720))
for frame in annotated:
    out.write(frame)
out.release()

In [ ]:
model = YOLO("runs/detect/train38/weights/best.pt")
detector = Detector(model, 0.5)
tracker = Tracker(detector)
video_path = r"C:\Users\Marcin\Videos\liverpool-real.mp4"
frames = load_frames(video_path)

In [ ]:
tracks = tracker.track(frames)

In [ ]:
teams = TeamAffiliation(tracks, frames)
teams.save_video(r"C:\Users\Marcin\Videos\liverpool-real_track.mp4")

In [ ]:
tracker.save_video(frames, r"C:\Users\Marcin\Videos\liverpool-real_track.mp4")